# Bibliotecas

In [1]:
import pandas as pd
import numpy as np

# Obtenção da base de dados

In [2]:
url = 'https://www.football-data.co.uk/new/BRA.csv'
dados = pd.read_csv(url)

In [3]:
dados.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89


In [4]:
dados.columns

Index(['Country', 'League', 'Season', 'Date', 'Time', 'Home', 'Away', 'HG',
       'AG', 'Res', 'PH', 'PD', 'PA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD',
       'AvgA'],
      dtype='object')

As colunas após a coluna 'Res' referen-se a probabilidades de sites de apostas. Para este estudo não são importantes, logo as descartarei

In [5]:
dados = dados.loc[:,'Country':'Res']

## Tradução dos conteúdos

In [6]:
traducao = {
    'Country': 'pais', 
    'League': 'campeonato', 
    'Season': 'temporada',
    'Date': 'data_partida',
    'Time': 'hora_partida',
    'Home': 'time_mandante',
    'Away': 'time_visitante',
    'HG': 'gols_mandante',
    'AG': 'gols_visitante',
    'Res': 'resultado'
}

In [7]:
dados.rename(columns=traducao, inplace=True)

In [8]:
dados.head()

,pais,campeonato,temporada,data_partida,hora_partida,time_mandante,time_visitante,gols_mandante,gols_visitante,resultado
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A


A coluna Resultado tem a seguinte lógica: SE o mandante ganhar então H, se o visitante ganhar então A senão D. Como meu objeto de estudo é o São Paulo FC contra o Botafogo S/A, a minha referência será sempre o meu time

## Separando a base de estudo

In [9]:
visitante = dados.time_visitante.isin(['Botafogo RJ', 'Sao Paulo']).values
mandante = dados.time_mandante.isin(['Botafogo RJ', 'Sao Paulo']).values
base_estudo = dados.loc[visitante & mandante].copy()
base_estudo.head()

,pais,campeonato,temporada,data_partida,hora_partida,time_mandante,time_visitante,gols_mandante,gols_visitante,resultado
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H
198,Brazil,Serie A,2012,31/08/2012,01:00,Sao Paulo,Botafogo RJ,4.0,0.0,H
541,Brazil,Serie A,2013,01/09/2013,20:00,Botafogo RJ,Sao Paulo,0.0,0.0,D
739,Brazil,Serie A,2013,24/11/2013,21:30,Sao Paulo,Botafogo RJ,1.0,1.0,D
766,Brazil,Serie A,2014,20/04/2014,20:00,Sao Paulo,Botafogo RJ,3.0,0.0,H


Agora vou traduzir

In [10]:
def resultado_jogo(x, time_target):
    if x.resultado == 'D':
        r = 'E'
        p = 1
    elif x.time_mandante == time_target and x.resultado == 'H':
        r = 'V'
        p = 3
    elif x.time_visitante == time_target and x.resultado == 'A':
        r = 'V'
        p = 3
    else:
        r = 'D'
        p = 0
    return pd.Series([r, p])

#def pontos_resultado_jogo(x, time_target)

In [11]:
base_estudo[['resultado_traduzido', 'pontos_ganhos']] = base_estudo.apply(resultado_jogo, args=('Sao Paulo',), axis=1)

In [12]:
base_estudo['mando'] = base_estudo['time_visitante'].apply(lambda x : 'Visitante' if x == 'Sao Paulo' else 'Mandante')

In [13]:
base_estudo.drop(columns='resultado', inplace=True)
base_estudo.head()

,pais,campeonato,temporada,data_partida,hora_partida,time_mandante,time_visitante,gols_mandante,gols_visitante,resultado_traduzido,pontos_ganhos,mando
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,D,0,Visitante
198,Brazil,Serie A,2012,31/08/2012,01:00,Sao Paulo,Botafogo RJ,4.0,0.0,V,3,Mandante
541,Brazil,Serie A,2013,01/09/2013,20:00,Botafogo RJ,Sao Paulo,0.0,0.0,E,1,Visitante
739,Brazil,Serie A,2013,24/11/2013,21:30,Sao Paulo,Botafogo RJ,1.0,1.0,E,1,Mandante
766,Brazil,Serie A,2014,20/04/2014,20:00,Sao Paulo,Botafogo RJ,3.0,0.0,V,3,Mandante


# Descrevendo os dados

## Resultados

In [14]:
base_estudo.resultado_traduzido.value_counts(normalize=True) * 100

V    52.941176
D    23.529412
E    23.529412
Name: resultado_traduzido, dtype: float64

No periodo foram 53% de vitórias, um aproveitamento até que bom

In [15]:
base_estudo.groupby('mando').resultado_traduzido.value_counts(normalize=True) * 100

mando      resultado_traduzido
Mandante   V                      62.500000
           E                      25.000000
           D                      12.500000
Visitante  V                      44.444444
           D                      33.333333
           E                      22.222222
Name: resultado_traduzido, dtype: float64

62,5% de vitórias como mandante e 44,4% como visitante. Ainda é um resultado muito bom!

## Pontos ganhos

In [16]:
base_estudo.pontos_ganhos.sum() / (base_estudo.shape[0] * 3) * 100

60.78431372549019

60% de aproveitamento dos pontos disputados

In [17]:
base_estudo.groupby('mando').pontos_ganhos.sum() / (base_estudo.groupby('mando').pontos_ganhos.count()*3) * 100

mando
Mandante     70.833333
Visitante    51.851852
Name: pontos_ganhos, dtype: float64

Impressionantes 71% de aproveitamento dos pontos em casa e 51% de aproveitamento dos pontos fora de casa

## Resultados ao longo das temporadas

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
cores_sao_paulo = ["#FE0000", "#000000", "#808080"]
paleta_cores_sao_paulo = sns.set_palette(sns.color_palette(cores_sao_paulo))

In [20]:
# Inserir gráficos de série temporal